In [2]:
import abc
import tensorflow as tf

import numpy as np

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

2022-12-10 11:28:17.846863: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [159]:
class SparkEnv(py_environment.PyEnvironment):

    def __init__(self):
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')
        self._observation_spec = array_spec.ArraySpec(
            shape=(8,), dtype=np.float32, name='observation')
        self._reward_spec = array_spec.ArraySpec(
            shape=(8,), dtype=np.int32, name='reward')
        self._discount_spec = array_spec.ArraySpec(
            shape=(), dtype=np.float32, name='discount')
        self._state = None
        self._reward = None
        self._discount = None
        self._episode_ended = False
        # ACTION
        # 0: DO NOTHING
        # 1: INCREASE MEMORY
        # -1: DECREASE MEMORY
        self._action = 0
        #  WHERE IS THE DEFINITION
        self.thpt_avg = 0
        self.qos_avg = 0
        # REWARD

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec
    
    def reward_spec(self):
        return self._reward_spec
    
    
    def time_step_spec(self):
        return ts.time_step_spec(self._observation_spec, self._reward_spec)

    def _reset(self):
        self._state = 0
        self._episode_ended = False
        return ts.restart(np.array([self._state], dtype=np.int32))

    def increaseMemory(self):
        self._action = 1
        # SEND DEF TO SPARK

    def decreaseMemory(self):
        self._action = -1
        # SEND DEF TO SPARK

    def doNothing(self):
        None

    def endEval(self):
        None

    def getSparkObservation(self):
        return tf.nest.map_structure(lambda x: np.zeros(x.shape, x.dtype),
                                     self._observation_spec)

    def getRewards(self):
        return tf.nest.map_structure(lambda x: np.zeros(x.shape, x.dtype),
                                     self._reward_spec)
    def getDiscount(self):
        return tf.nest.map_structure(lambda x: np.zeros(x.shape, x.dtype),
                                     self._discount_spec)

    def _step(self, action):

        if self._episode_ended:
            # The last action ended the episode. Ignore the current action and start
            # a new episode.
            return self.reset()

        # TAKE ACTION
        if action == 1:
            self.increaseMemory()
        if action == -1:
            self.decreaseMemory()
        elif action == 0:
            self.doNothing()
        else:
            raise ValueError('`action` should be -1, 0 or 1.')

        # WAIT FOR OBSERVATION FROM SPARK
        # Simulate Random
        self.state = self.getSparkObservation()
        self._state = self.state
        thpttot, thptvar, tproc, tsche, msgsspark, msgsgb, memready, qosthresh = self.getSparkObservation()
        # REWARD

        self._reward = self.getRewards()
        # return ts.transition(
        #     np.array([self._state], dtype=np.int32),
        #     self.reward=0.0,
        #     self._episode_ended,
        #     False,
        #     {})

        self._discount = self.getDiscount()

        return ts.TimeStep(
            # array_spec.ArraySpec(
            #     self.state, dtype=np.int32, name='observation'),
            observation=self._state,
            reward=self._reward,
            # self._episode_ended,
            discount=self._discount,
            step_type=None
        )


In [168]:
env = SparkEnv()

one = np.array(0, dtype=np.int32)
zero = np.array(1, dtype=np.int32)

env._step(one)
# env._reward_spec
# env._observation_spec
# env._action_spec
# env.time_step_spec()

TimeStep(
{'discount': array(0., dtype=float32),
 'observation': array([0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 'reward': array([0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 'step_type': None})

In [169]:
env = SparkEnv()
utils.validate_py_environment(env, episodes=5)

ValueError: Given `time_step`: TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([0], dtype=int32),
 'reward': array(0., dtype=float32),
 'step_type': array(0, dtype=int32)}) does not match expected `time_step_spec`: TimeStep(
{'discount': BoundedArraySpec(shape=(), dtype=dtype('float32'), name='discount', minimum=0.0, maximum=1.0),
 'observation': ArraySpec(shape=(8,), dtype=dtype('float32'), name='observation'),
 'reward': ArraySpec(shape=(8,), dtype=dtype('int32'), name='reward'),
 'step_type': ArraySpec(shape=(), dtype=dtype('int32'), name='step_type')})